# 트랜스포머로 만드는 대화형 챗봇
-----
### 프로젝트 - 한국어 데이터로 챗봇 만들기

트랜스포머 모델을 기반으로 한 인코더-디코더 구조를 바탕으로 챗봇을 제작해 보려고 합니다.<br>
많은 자연어 처리 모델들은 텍스트 문장을 입력으로 받기 위해 단어를 임베딩 벡터로 변환하는 벡터화 과정을 거칩니다. 트랜스포머 또한 그 점에서는 다른 모델들과 다르지 않습니다.<br>하지만 트랜스포머 모델의 입력데이터 처리에는 RNN 계열의 모델들과 다른 점이 한 가지 있습니다. 바로 임베딩 벡터에 어떤 값을 더해준 뒤에 입력으로 사용한다는 점입니다.<br>그 값은 바로 아래의 **`포지셔널 인코딩(positional Encoding)`**에 해당하는 부분입니다.![](https://aiffelstaticprd.blob.core.windows.net/media/original_images/Untitled_4_fuzN6PD.png)<br>
**트랜스포머가 RNN과 결정적으로 다른 점이 바로 이 부분입니다.**<br>문장에 있는 모든 단어를 한꺼번에 문장 단위로 입력받는 트랜스포머는 자칫 'I ate lunch'와 'lunch ate I'를 구분할 수 없을지도 모릅니다.<br>그래서 같은 단어라도 그 단어가 문장의 몇 번째 어순으로 입력되었는지를 모델에 추가로 알려 주기 위해, 단어의 임베딩 벡터에다가 위치 정보를 가진 벡터(Positional Encoding) 값을 더해서 모델의 입력으로 삼는 것이지요!

## 실습목표
>* 트랜스포머의 인코더 디코더 구조 이해하기
>* 내부 단어 토크나이저 사용하기
>* 셀프 어텐션 이해하기
>* 한국어에도 적용해보기

## 순서
> **Step1.** 데이터 수집하기<br>
**Step2.** 데이터 전처리하기<br>
**Step3.** SubwordTextEncoder 사용하기<br>
**Step4.** 모델 구성하기<br>
**Step5.** 모델 평가하기<br>

## 간단정리! 
---
참고자료 : [챗봇의 5가지 대표 유형](https://tonyaround.com/%ec%b1%97%eb%b4%87-%ea%b8%b0%ed%9a%8d-%eb%8b%a8%ea%b3%84-%ec%b1%97%eb%b4%87%ec%9d%98-5%ea%b0%80%ec%a7%80-%eb%8c%80%ed%91%9c-%ec%9c%a0%ed%98%95-%ec%a2%85%eb%a5%98/)
- **대화형 챗봇** : 자연어처리를 기반으로 자연스럽게 대화가 가능(머신러닝 및 딥러닝을 기본)<br>
- **트리형(버튼) 챗봇** : 정해진 트리구조를 따라 답변을 얻는 형태, 챗봇의 1세대에서 많이 썼던 방식(인공지능 X)
![트리형 챗봇](https://i2.wp.com/tonyaround.com/wp-content/uploads/2019/09/unnamed-file.png?resize=1024%2C533&ssl=1)

- **추천형 챗봇** : 표면적으로는 대화의 형태를 띄고 있지만, 답변을 제공하는 방식이 대화형 챗봇과 다름, 대화형 챗봇으로 가기 위한 중간 과정으로도 사용
![추천형 챗봇](https://i2.wp.com/tonyaround.com/wp-content/uploads/2019/09/unnamed-file-2.png?w=695&ssl=1)

- **시나리오형 챗봇** : 원하는 서비스 혹은 아웃풋 제공을 위하여 정해진 시나리오를 수행해주는 챗봇, 가장 투자대비 효과가 좋은 챗봇 유형
![시나리오형 챗봇](https://i2.wp.com/tonyaround.com/wp-content/uploads/2019/09/unnamed-file-3.png?w=893&ssl=1)

- **결합형 챗봇** : 비즈니스 목적에 따라 위의 챗봇 유형들을 결합해서 설계가 가능
![결합형 챗봇](https://i2.wp.com/tonyaround.com/wp-content/uploads/2019/09/unnamed-file-4.png?resize=1024%2C557&ssl=1)

대화형을 제외하면 사실상 챗봇은 대화형 UX를 가졌지만 본질적으로는 검색엔진이거나, 혹은 음성ARS를 대화형 UX에 옮겨놓은 것이라 할 수 있습니다.


### 인코더와 디코더 구조 되짚어보기
---
![인코더와 디코더 구조](https://aiffelstaticprd.blob.core.windows.net/media/images/Untitled_UcFQAjh.max-800x600.png)

번역기를 만드는 데 사용한 대표적인 모델인 인코더와 디코더 구조를 되짚어 봅시다.<br>인코더에 입력 문장이 들어가고, 디코더는 이에 상응하는 출력 문장을 생성합니다.<br>이를 훈련한다는 것은 결국 입력 문장과 출력 문장 두 가지 병렬 구조로 구성된 데이터셋을 훈련한다는 의미였습니다.

**✓ 훈련 데이터셋의 구성(번역)**
- 입력 문장 : '저는 학생입니다.'
- 출력 문장 : 'I am a student'

**✓ 훈련 데이터셋의 구성(질문-답변)**
- 입력 문장 : '오늘의 날씨는 어때?'
- 출력 문장 : '오늘은 매우 화창한 날씨야'

### 트랜스포머의 인코더와 디코더
---
마찬가지로 입력 문장을 넣으면 출력 문장을 내뱉는 구조입니다.
![트랜스포머의 인코더와 디코더 구조](https://aiffelstaticprd.blob.core.windows.net/media/images/Untitled_1_kxflIxg.max-800x600.png)<br>

위의 블랙박스로 가려져 있는 트랜스포머의 내부구조를 열어보면 아래와 같습니다!
![내부구조](https://aiffelstaticprd.blob.core.windows.net/media/images/Untitled_2_EnQyi4S.max-800x600.png)<br>
그리고 그 내부를 조금 더 확대해 보면 아래와 같이 톱니바퀴처럼 맞물려 돌아가는 여러 가지 부품들로 구성되어져 있습니다.
![세부내부구조](https://aiffelstaticprd.blob.core.windows.net/media/images/Untitled_3_ddZedfW.max-800x600.png)

## 준비물
---
터미널을 열어 실습에 필요한 디렉토리를 생성합니다.

> $ mkdir -p ~/aiffel/songys_chatbot

### Step1. 데이터 수집하기
---
한국어 챗봇 데이터는 송영숙님이 공개한 챗봇 데이터를 사용합니다.

이 데이터는 아래의 링크에서 다운로드할 수 있습니다.

[songys/Chatbot_data](https://github.com/songys/Chatbot_data/blob/master/ChatbotData%20.csv)

**`wget으로 데이터 다운로드`**

>$ wget https://github.com/songys/Chatbot_data/raw/master/ChatbotData%20.csv

**✓ 사용할 패키지(라이브러리)들을 import 해보겠습니다.**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
import time
import tensorflow_datasets as tfds
import tensorflow as tf

Duplicate key in file PosixPath('/home/aiffel-dj49/anaconda3/envs/aiffel/lib/python3.7/site-packages/matplotlib/mpl-data/matplotlibrc'), line 250 ('font.family: NanumGothic')


In [4]:
data = pd.read_csv('ChatbotData .csv')

FileNotFoundError: [Errno 2] No such file or directory: 'ChatbotData .csv'

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData%20.csv", filename="ChatBotData.csv")
train_data = pd.read_csv('ChatBotData.csv')
train_data.head()

In [ ]:
print('챗봇 샘플의 개수 :', len(train_data))

### Step2. 데이터 전처리하기
---
영어 데이터와는 전혀 다른 데이터인 만큼 영어 데이터에 사용했던 전처리와 일부 동일한 전처리도 필요하겠지만 전체적으로는 다른 전처리를 수행해야 할 수도 있습니다.<br>이번 전처리는 정규 표현식(Regular Expression)을 사용하여 구두점(punctuation)을 제거하여 단어를 토크나이징(tokenizing)하는 일에 방해가 되지 않도록 정제하는 것을 목표로 합니다.

In [ ]:
# 사용할 샘플의 최대 개수
MAX_SAMPLES = 50000
print('사용할 샘플의 최대 개수', MAX_SAMPLES, '개')

In [ ]:
# 전처리 함수
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()

    # 단어와 구두점(punctuation) 사이의 거리를 만듭니다.
    # 예를 들어서 "I am a student." => "I am a student ."와 같이
    # student와 온점 사이에 거리를 만듭니다.
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)

    # (a-z, A-Z, ".", "?", "!", ",")를 제외한 모든 문자를 공백인 ' '로 대체합니다.
    sentence = re.sub(r"[^a-zA-Z?.!,]+", " ", sentence)
    sentence = sentence.strip()
    return sentence

In [ ]:
# 질문과 답변의 쌍인 데이터셋을 구성하기 위한 데이터 로드 함수
def load_conversations():
    id2line = {}
    with open(path_to_movie_lines, errors='ignore') as file:
        lines = file.readlines()
    for line in lines:
        parts = line.replace('\n', '').split(' +++$+++ ')
        id2line[parts[0]] = parts[4]

    inputs, outputs = [], []
    with open(path_to_movie_conversations, 'r') as file:
        lines = file.readlines()

    for line in lines:
        parts = line.replace('\n', '').split(' +++$+++ ')
        conversation = [line[1:-1] for line in parts[3][1:-1].split(', ')]

    for i in range(len(conversation) - 1):  # 전처리 함수를 질문에 해당되는 inputs와 답변에 해당되는 outputs에 적용.
        inputs.append(preprocess_sentence(id2line[conversation[i]]))
        outputs.append(preprocess_sentence(id2line[conversation[i + 1]]))

        if len(inputs) >= MAX_SAMPLES:
            return inputs, outputs
    return inputs, outputs

In [ ]:
# 데이터를 로드하고 전처리하여 질문을 questions, 답변을 answers에 저장합니다.
questions, answers = load_conversations()
print('전체 샘플 수 :', len(questions), '개')
print('전체 샘플 수 :', len(answers), '개')

### Step3. SubwordTextEncoder 사용하기
---
한국어 데이터는 형태소 분석기를 사용하여 토크나이징을 해야 한다고 많은 분이 알고 있습니다.<br>하지만 여기서는 형태소 분석기가 아닌 위 실습에서 사용했던 내부 단어 토크나이저인 **`SubwordTextEncoder`**를 그대로 사용해보세요.

1. TensorFlow Datasets **SubwordTextEncoder**를 토크나이저로 사용한다.  단어보다 더 작은 단위인 Subword를 기준으로 토크나이징하고,  각 토큰을 고유한 **정수로 인코딩**한다.
2. 각 문장을 토큰화하고 각 문장의 시작과 끝을 나타내는 **`START_TOKEN`** 및 **`END_TOKEN`**을 추가한다.
3. 최대 길이 **MAX_LENGTH**인 40을 넘는 문장들은 필터링한다.
4. MAX_LENGTH보다 길이가 짧은 문장들은 40에 맞도록 **패딩**한다.

#### **✓ 단어장(Vocabulary) 만들기**
---
우선 각 단어에 고유한 정수 인덱스를 부여하기 위해서 단어장(Vocabulary)을 만들어보겠습니다.<br>단어장을 만들 때는 질문과 답변 데이터셋을 모두 사용하여 만듭니다.

In [ ]:
# tensorflow 버전 출력
print(tf.__version__)

In [ ]:
print("살짝 오래 걸릴 수 있어요. 스트레칭 한 번 해볼까요? 👐")

# 질문과 답변 데이터셋에 대해서 Vocabulary 생성. (Tensorflow 2.2.0 이하)
tokenizer = tfds.features.text.SubwordTextEncoder.build_from_corpus(questions + answers, target_vocab_size=2**13)

# (주의) Tensorflow 2.3.0 이상의 버전에서는 아래 주석의 코드를 대신 실행해 주세요. 
#tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(questions + answers, target_vocab_size=2**13)

In [ ]:
# 시작 토큰과 종료 토큰에 고유한 정수를 부여
# 단어장의 번호와 겹치지 않도록 각각 단어장의 크기와 그보다 1이 큰 수를 번호로 부여
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]

In [ ]:
# 시작 토큰과 종료 토큰에 부여된 정수를 출력
print('START_TOKEN의 번호 :' ,[tokenizer.vocab_size])
print('END_TOKEN의 번호 :' ,[tokenizer.vocab_size + 1])

각각 8,331과 8,332라는 점에서 현재 단어장의 크기가 8,331(0번부터 8,330번)이라는 의미입니다.

In [ ]:
# 시작 토큰과 종료 토큰을 고려하여 +2를 하여 단어장의 크기를 산정합니다.
VOCAB_SIZE = tokenizer.vocab_size + 2

print(VOCAB_SIZE)

#### **✓ 각 단어를 고유한 정수로 인코딩(Integer encoding) & 패딩(Padding)**
---
위에서 **`tensorflow_datasets`**의 **`SubwordTextEncoder`**를 사용해서 tokenizer를 정의하고 Vocabulary를 만들었다면, **`tokenizer.encode()`**로 각 단어를 정수로 변환할 수 있고 또는 **`tokenizer.decode()`**를 통해 정수 시퀀스를 단어 시퀀스로 변환할 수 있습니다.

In [ ]:
# 임의의 22번째 샘플에 대해서 정수 인코딩 작업을 수행.
# 각 토큰을 고유한 정수로 변환
print('정수 인코딩 후의 21번째 질문 샘플: {}'.format(tokenizer.encode(questions[21])))
print('정수 인코딩 후의 21번째 답변 샘플: {}'.format(tokenizer.encode(answers[21])))

각 단어에 고유한 정수가 부여된 Vocabulary를 기준으로 단어 시퀀스가 정수 시퀀스로 인코딩된 결과를 확인할 수 있습니다. 위의 결과와 마찬가지로 질문과 답변 셋에 대해서 전부 **정수 인코딩**을 수행합니다. 이와 동시에 문장의 최대 길이를 정하고, 해당 길이로 **패딩(padding)**합니다.

In [ ]:
# 샘플의 최대 허용 길이 또는 패딩 후의 최종 길이
MAX_LENGTH = 40
print(MAX_LENGTH)

In [ ]:
# 정수 인코딩, 최대 길이를 초과하는 샘플 제거, 패딩
def tokenize_and_filter(inputs, outputs):
    tokenized_inputs, tokenized_outputs = [], []
  
    for (sentence1, sentence2) in zip(inputs, outputs):
    # 정수 인코딩 과정에서 시작 토큰과 종료 토큰을 추가
    sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
    sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN

    # 최대 길이 40 이하인 경우에만 데이터셋으로 허용
    if len(sentence1) <= MAX_LENGTH and len(sentence2) <= MAX_LENGTH:
        tokenized_inputs.append(sentence1)
        tokenized_outputs.append(sentence2)
  
    # 최대 길이 40으로 모든 데이터셋을 패딩
    tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
    tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(tokenized_outputs, maxlen=MAX_LENGTH, padding='post')

    return tokenized_inputs, tokenized_outputs

In [ ]:
# 단어장의 크기와 샘플의 개수를 확인
questions, answers = tokenize_and_filter(questions, answers)
print('단어장의 크기 :',(VOCAB_SIZE))
print('필터링 후의 질문 샘플 개수: {}'.format(len(questions)), '개')
print('필터링 후의 답변 샘플 개수: {}'.format(len(answers)), '개')

#### **✓ 교사 강요(Teacher Forcing) 사용하기**
---
디코더의 입력과 실제값(레이블)을 정의해주기 위해서는 **교사 강요(Teacher Forcing)**이라는 언어 모델의 훈련 기법을 이해해야만 합니다. 질문과 답변의 쌍을 **tf.data.Dataset API**의 입력으로 사용하여 파이프라인을 구성합니다. 이때, 교사 강요를 위해서 **`answers[:, :-1]`**를 디코더의 입력값, **`answers[:, 1:]`**를 디코더의 레이블로 사용합니다.

In [ ]:
BATCH_SIZE = 64
BUFFER_SIZE = 20000

# 디코더는 이전의 target을 다음의 input으로 사용합니다.
# 이에 따라 outputs에서는 START_TOKEN을 제거하겠습니다.
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': questions,
        'dec_inputs': answers[:, :-1]
    },
    {
        'outputs': answers[:, 1:]
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

### Step4. 모델 구성하기
---
위 실습 내용을 참고하여 트랜스포머 모델을 구현합니다.

In [ ]:
def transformer(vocab_size,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                name="transformer"):
    inputs = tf.keras.Input(shape=(None,), name="inputs")
    dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

	# 인코더에서 패딩을 위한 마스크
    enc_padding_mask = tf.keras.layers.Lambda(
        create_padding_mask, output_shape=(1, 1, None),
        name='enc_padding_mask')(inputs)

  # 디코더에서 미래의 토큰을 마스크 하기 위해서 사용합니다.
  # 내부적으로 패딩 마스크도 포함되어져 있습니다.
    look_ahead_mask = tf.keras.layers.Lambda(
        create_look_ahead_mask,
        output_shape=(1, None, None),
        name='look_ahead_mask')(dec_inputs)

  # 두 번째 어텐션 블록에서 인코더의 벡터들을 마스킹
  # 디코더에서 패딩을 위한 마스크
    dec_padding_mask = tf.keras.layers.Lambda(
        create_padding_mask, output_shape=(1, 1, None),
        name='dec_padding_mask')(inputs)

  # 인코더
  enc_outputs = encoder(
      vocab_size=vocab_size,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
  )(inputs=[inputs, enc_padding_mask])

  # 디코더
  dec_outputs = decoder(
      vocab_size=vocab_size,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
  )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

    # 완전연결층
    outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)

**✓ 모델 생성**

In [ ]:
# 원활한 훈련을 위해 논문에서보다는 작은 값을 사용
tf.keras.backend.clear_session()

# 하이퍼파라미터
NUM_LAYERS = 2 # 인코더와 디코더의 층의 개수
D_MODEL = 256 # 인코더와 디코더 내부의 입, 출력의 고정 차원
NUM_HEADS = 8 # 멀티 헤드 어텐션에서의 헤드 수 
UNITS = 512 # 피드 포워드 신경망의 은닉층의 크기
DROPOUT = 0.1 # 드롭아웃의 비율

model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

model.summary()

**✓ 손실 함수(Loss function)**

In [ ]:
def loss_function(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))

    loss = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction='none')(y_true, y_pred)

    mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
    loss = tf.multiply(loss, mask)

    return tf.reduce_mean(loss)

#### **✓ 커스텀된 학습률(Learning rate) : 매우 중요한 하이퍼파라미터**
---
- **커스텀 학습률 스케줄링(Custom Learning rate Scheduling)** : 모델학습 초기에 learning rate를 급격히 높였다가, 이후 train step이 진행됨에 따라 서서히 낮추어 가면서 안정적으로 수렴하게 하는 고급 기법

In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps**-1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [ ]:
# 커스텀 학습률 스케줄링 계획을 시각화
sample_learning_rate = CustomSchedule(d_model=128)

plt.plot(sample_learning_rate(tf.range(200000, dtype=tf.float32)))
plt.ylabel("Learning Rate")
plt.xlabel("Train Step")

**✓ 모델 컴파일 : 손실 함수와 커스텀 된 학습률(learning rate)을 사용하여 모델을 컴파일**

In [ ]:
learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
    return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

**✓ 훈련하기 : 총 20 에포크를 학습**

In [ ]:
EPOCHS = 20
model.fit(dataset, epochs=EPOCHS, verbose=1)

### Step5. 모델 평가하기
---
Step 1에서 선택한 전처리 방법을 고려하여 입력된 문장에 대해서 대답을 얻는 예측 함수를 만듭니다. 예측(inference) 단계는 기본적으로 다음과 같은 과정을 거칩니다.

1. 새로운 입력 문장에 대해서는 훈련 때와 동일한 전처리를 거친다.
2. 입력 문장을 토크나이징하고, **`START_TOKEN`**과 **`END_TOKEN`**을 추가한다.
3. 패딩 마스킹과 룩 어헤드 마스킹을 계산한다.
4. 디코더는 입력 시퀀스로부터 다음 단어를 예측한다.
5. 디코더는 예측된 다음 단어를 기존의 입력 시퀀스에 추가하여 새로운 입력으로 사용한다.
6. **`END_TOKEN`**이 예측되거나 문장의 최대 길이에 도달하면 디코더는 동작을 멈춘다.<br>
위의 과정을 모두 담은 **`decoder_inference()`** 함수를 만듭니다.

In [ ]:
def decoder_inference(sentence):
    sentence = preprocess_sentence(sentence)

    # 입력된 문장을 정수 인코딩 후, 시작 토큰과 종료 토큰을 앞뒤로 추가.
    # ex) Where have you been? → [[8331   86   30    5 1059    7 8332]]
    sentence = tf.expand_dims(
        START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

    # 디코더의 현재까지의 예측한 출력 시퀀스가 지속적으로 저장되는 변수.
    # 처음에는 예측한 내용이 없음으로 시작 토큰만 별도 저장. ex) 8331
    output_sequence = tf.expand_dims(START_TOKEN, 0)

    # 디코더의 인퍼런스 단계
    for i in range(MAX_LENGTH):
        # 디코더는 최대 MAX_LENGTH의 길이만큼 다음 단어 예측을 반복합니다.
        predictions = model(inputs=[sentence, output_sequence], training=False)
        predictions = predictions[:, -1:, :]

        # 현재 예측한 단어의 정수
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

        # 만약 현재 예측한 단어가 종료 토큰이라면 for문을 종료
        if tf.equal(predicted_id, END_TOKEN[0]):
            break

        # 예측한 단어들은 지속적으로 output_sequence에 추가됩니다.
        # 이 output_sequence는 다시 디코더의 입력이 됩니다.
        output_sequence = tf.concat([output_sequence, predicted_id], axis=-1)

    return tf.squeeze(output_sequence, axis=0)

임의의 입력 문장에 대해서 **`decoder_inference()`** 함수를 호출하여 챗봇의 대답을 얻는 **`sentence_generation()`** 함수를 만듭니다.

In [ ]:
def sentence_generation(sentence):
    # 입력 문장에 대해서 디코더를 동작 시켜 예측된 정수 시퀀스를 리턴받습니다.
    prediction = decoder_inference(sentence)

    # 정수 시퀀스를 다시 텍스트 시퀀스로 변환합니다.
    predicted_sentence = tokenizer.decode(
      [i for i in prediction if i < tokenizer.vocab_size])

    print('입력 : {}'.format(sentence))
    print('출력 : {}'.format(predicted_sentence))

    return predicted_sentence

그럼 이제 임의의 문장으로부터 챗봇의 대답을 얻어봅시다!

In [ ]:
sentence_generation('나랑 눈사람 만들래?')

In [ ]:
sentence_generation("다 잊어~ 다 잊어~")

# 루브릭 평가 기준
---
번호|평가문항|상세기준
---|---|---
1 |한국어 전처리를 통해 학습 데이터셋을 구축하였다.|공백과 특수문자 처리, 토크나이징, 병렬데이터 구축의 과정이 적절히 진행되었다.
2 |트랜스포머 모델을 구현하여 한국어 챗봇 모델 학습을 정상적으로 진행하였다.|구현한 트랜스포머 모델이 한국어 병렬 데이터 학습 시 안정적으로 수렴하였다.
3 |한국어 입력문장에 대해 한국어로 답변하는 함수를 구현하였다.|한국어 입력문장에 그럴듯한 한국어로 답변을 리턴하였다.

# 회고
---